## Calculating Index in a 2D image


```c
int row = blockIdx.y * blockDim.y + threadIdx.y;
int col = blockIdx.x * blockDim.x + threadIdx.x;
int plane = blockIdx.z * blockDim.z + threadIdx.z;

int matrixSize = blockDim.x * gridDim.x;

int offset = plane * matrixSize 
if (ROW_MAJOR) {
    offset += row * blockDim.x + col;
} else if (COL_MAJOR) {
    offset += col * blockDim.y + row;
}
```


### BLAS

Level 1: Vector-vector operations
Level 2: Matrix-vector operations
Level 3: Matrix-matrix operations



# Solutions

# 1. 2 MatMul kernels
In this chapter we implemented a matrix multiplication kernel that has each
thread produce one output matrix element. In this question, you will
implement different matrix-matrix multiplication kernels and compare them.

- Write a kernel that has each thread produce one output matrix row. Fill in the execution configuration parameters for the design.
- Write a kernel that has each thread produce one output matrix column. Fill in the execution configuration parameters for the design.
- Analyze the pros and cons of each of the two kernel designs.


```c
// Textbook GEMM, Square matrix
// 1 kernel produces 1 output element
__global__ void matmulkernel_0(float* M, float* N, float* P, int width) {
  int row = threadIdx.y + blockIdx.y * blockDim.y;
  int col = threadIdx.x + blockIdx.x * blockDim.x;

  if ((row < width) && (col < width)) {
    float pVal = 0;
      for (int k = 0; k < width; k++)
          pVal += M[row * width + k] * N[k * width + col];
        P[row * width + col] = pVal;
  }
}

// Host code
dim3 threadsPerBlock(16, 16);
dim3 blocksPerGrid((n + 15) / 16, (n + 15) / 16);
matMulKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, n);

// 1 kernel produces 1 output row
__global__ void matmulkernel_1(float* M, float* N, float* P, int width) {
  int row = threadIdx.y + blockIdx.y * blockDim.y;
  if (row < width) {
    for (int col = 0; col < width; col++) {
      float pVal = 0;
      for (int k = 0; k < width; k++)
        pVal = M[row * width + k] + N[k * width + col];
      P[row * width + col] = pVal;
    }
  }
}

// 1 kernel produces 1 output column
__global__ void matmulkernel_2(float* M, float* N, float* P, int width) {
  int col = threadIdx.x + blockIdx.x * blockDim.x;

  if (col < width) {
    for (int row = 0; row < width; row++) {
      float pVal = 0;
      for (int k = 0; k < width; k++)
        pVal = M[row * width + k] + N[k * width + col];
      P[row * width + col] = pVal;
    }
  }
}
int threadsPerBlock(256);
int blocksPerGrid((n + threadsPerBlock - 1) / threadsPerBlock);
matMulKernel<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, n);
```


## 2. MatMul kernel

```c
__global__ void matmulKernel(float* A /*Out Vector*/, float* B/*In Matrix*/, float* C/*In Vector*/, int width) {
  int i = threadIdx.x + blockIdx.x * blockDim.x;
  float sum = 0;
  if (i < width) {
    for (int j = 0; j < width; j++)
      sum += B[i * width + j] * C[j];
    A[i] = sum;
  }
}


void matMulHost(float* h_A, float* h_B, float* h_C, int vectorLen) {
  float *d_A, *d_C; // vector
  float *d_B, // matrix
  int vecSize = sizeof(float) * vectorLen;
  int matSize = vecSize * vectorLen;
  cudaMalloc((void**)&d_A, vecSize);
  cudaMalloc((void**)&d_C, vecSize);
  cudaMalloc((void**)&d_B, matSize);
  cudaMemset(d_A, 0, vecSize);
  cudaMemcpy(d_B, h_B, matSize, cudaMemcpyHostToDevice);
  cudaMemcpy(d_C, h_C, vecSize, cudaMemcpyHostToDevice);
  
  dim3 threads(128);
  dim3 blocks((vectorLen + threads - 1) / threads);
  matmulKernel<<<blocks, threads>>>(d_A, d_B, d_C, n);

  cudaFree(d_B);
  cudaFree(d_C);
  cudaMemcpy(h_A, d_A, vecSize, cudaMemcpyDeviceToHost);
  cudaFree(d_A);
}

```

## 3. CUDA Kernel

In [12]:
N, M = 300, 150
T_Y, T_X = 16, 32
threadsPerBlock = T_X * T_Y
Blocks = ((N-1)//T_Y + 1) * ((M-1)//T_X + 1)
threadsInGrids = threadsPerBlock * Blocks
threadsUseful = N * M
threadsPerBlock, threadsInGrids, Blocks, threadsUseful, threadsUseful/threadsInGrids*100


(512, 48640, 95, 45000, 92.51644736842105)

## 4. 

In [14]:
W, H = 400, 500
row, col = 20, 10

rowMajorOffset = row * W + col
colMajorOffset = col * H + row

rowMajorOffset, colMajorOffset

(8010, 5020)

## 5.

In [21]:
# Consider a 3D tensor with a width of 400, a height of 500, and a depth of
# 300. The tensor is stored as a one-dimensional array in row-major order.
# Specify the array index of the tensor element at x5 10, y5 20, and z5 5.

W = 400
H = 500
D = 300

x = 10
y = 20
z = 5

rowMajorOffset = (W * H * z) + (y * W) + x
rowMajorOffset

1008010